# Exercice: calculer la distance à laquelle un projectile touche le sol

**Version originale du notebook par: Giovanni Pizzi, EPFL.    Adapté par: Nathaniel Raimbault.**

Dans cet exercice, on considère trois paramètres régissant les conditions initiales de lancé d'un projectile, qui sont:

- la hauteur $h$ à partir de laquelle le projectile est lancé.
- les deux composantes (horizontale $v_x$ et verticale $v_y$) de la vitesse, $\vec v = (v_x, v_y)$ à laquelle le projectile est initialement jeté.

![Schéma explicatif](../images/explanation.png)

## Tâche
**Votre but est ici d'écrire une fonction Python qui prend en entrée (input) les trois paramètres ci-dessus, et qui renvoie en sortie (output) la distance au sol parcourue quand le projectile touche le sol, lorsqu'on néglige les frottements fluides dus à l'air.**


## Comment tester les résultats
Pour tester votre fonction, faites bouger les curseurs ci-dessous; Ceux-ci permettent de faire varier les conditions initiales de lancement.

Une visualisation en temps réel vous montrera l'allure réelle de la trajectoire en trait plein bleu (absence de frottements) ou en pointillés noirs (avec frottements), le point de lancement étant représenté par un disque noir et le point de contact au sol par une croix bleue ou noire. Le résultat de votre fonction sera quant à lui représenté par un cercle rouge.

Enfin, en ouvrant l'onglet "Résultats de validation de votre fonction", vous pourrez analyser vos potentielles erreurs.

In [1]:
%matplotlib notebook
import numpy as np
import pylab as pl
from scipy.optimize import fsolve

import tabulate
from ipywidgets import Label, Button, Output, FloatSlider, HBox, VBox, Layout, HTML, Accordion, Dropdown
from widget_code_input import WidgetCodeInput
from IPython.display import display

In [2]:
# Value of the vertical (downwards) acceleration
g = 9.81 # m/s^2

In [3]:
code_widget = WidgetCodeInput(
    function_name="get_hit_coordinate", 
    function_parameters="vertical_position, horizontal_v, vertical_v, g",
    docstring="""
Une fonction pour calculer la distance totale parcourue par un projectile lancé en l'air, 
en négligeant les frottements, en fonction des conditions initiales du lancé, lorsque les frottements sont négligés.

:param vertical_position: hauteur initiale du projectile [m]
:param horizontal_v: vitesse horizontale initiale [m/s]
:param vertical_v: vitesse verticale initiale [m/s] 
    (une valeur positive traduit une vitesse ascendante)
:param g: l'accélération verticale (descendante) (par défaut: pesanteur terrestre)
    
:renvoie: la position à laquelle le projectile touchera le sol [m]
""",
    function_body="# Ecris ici ta solution\n# Après avoir modifié la fonction, fais bouger l'un des curseurs afin de valider ta fonction")
display(code_widget)

## The solution:
# import math
# return horizontal_v * (vertical_v + math.sqrt(vertical_v**2 + 2. * g * vertical_position)) / g

WidgetCodeInput(code_theme='nord', docstring="\nUne fonction pour calculer la distance totale parcourue par un…

In [4]:
vertical_position_widget = FloatSlider(
    value=6, min=0, max=15, 
    description="Hauteur initiale [m]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
horizontal_v_widget = FloatSlider(
    value=5, min=-10, max=10, 
    description="Vitesse horizontale initiale [m/s]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
vertical_v_widget = FloatSlider(
    value=3, min=-20, max=20, 
    description="Vitesse verticale initiale [m/s]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))
g_widget = Dropdown(options=(
        ("Terre",9.81),
        ("Lune",1.62),
        ("Mars",3.71),
        ("Jupyter",24.79),
    ), 
    description = "Gravité [m/s^2]", continuous_update=False, layout=Layout(width='50%',min_width='200px'))
friction_widget = FloatSlider(
    value=0, min=0, max=0.1, step=0.005, 
    description="Frottements [kg/s]",
    continuous_update=False, 
    style={'description_width': 'initial'}, layout=Layout(width='50%', min_width='350px'))


plot_box = Output()

input_box = VBox([vertical_position_widget, horizontal_v_widget, vertical_v_widget, g_widget, friction_widget])
display(HBox([input_box, plot_box]))

In [5]:
check_function_output = Output()
check_accordion = Accordion(children=[check_function_output], selected_index=None)
check_accordion.set_title(0, 'Résultats de la validation de ta fonction (clique pour les afficher)')

display(check_accordion)

Accordion(children=(Output(),), selected_index=None, _titles={'0': 'Résultats de la validation de ta fonction …

In [6]:
def trajectory(t, vertical_position, horizontal_v, vertical_v, g):
    """
    Return the coordinates (x, y) at time t
    """
    # We define the initial x coordinate to be zero
    x0 = 0
        
    x = x0 + horizontal_v * t
    y = -0.5 * g* t**2 + vertical_v * t + vertical_position
    
    return x, y

def hit_conditions(vertical_position, horizontal_v, vertical_v, g):
    """
    Return (t, D), where t is the time at which the ground is hit, and D 
    is the distance at which the projectile hits the ground
    """
    
    # We define the initial x coordinate to be zero
    x0 = 0
    
    # x = x0 + horizontal_v * t => t = (x-x0) / horizontal_v
    # y = -0.5 * g* t**2 + vertical_v * t + vertical_position => 
    #
    # y == 0 => 
    a = -0.5 * g
    b = vertical_v
    c = vertical_position
    
    # the two solutions; I want the solution with positive t, 
    # that will in any case be t1, because
    # t1 > t2 for any value of a, b, c (since a < 0)
    t1 = (-b - np.sqrt(b**2 - 4 * a * c)) / (2. * a)
    #t2 = (-b + np.sqrt(b**2 - 4 * a * c)) / (2. * a)
    
    t = t1
    
    D = x0 + horizontal_v * t
    
    return t, D


In [7]:
def trajectory_friction(t, vertical_position, horizontal_v, vertical_v, g, alpha):
    """
    Return the coordinates (x, y) at time t
    """
    # We define the initial x coordinate to be zero
    x0 = 0
    
    m = 0.1 # in kg
    #alpha = 3*10**(-3) # in kg/s
    tau = m/alpha
    vylim = -g*m/alpha 
        
    x = x0 + tau*horizontal_v * (1-np.exp(-t/tau))
    y = vertical_position + tau*(vertical_v-vylim)*(1-np.exp(-t/tau)) + vylim * t
    
    return x, y

def hit_conditions_friction(vertical_position, horizontal_v, vertical_v, g, alpha):
    """
    Return (Tv, D), where Tv is the time at which the ground is hit, and D 
    is the distance at which the projectile hits the ground, in case friction is taken into account.
    """
    
    # We define the initial x coordinate to be zero
    x0 = 0
    # We define the mass, which is here hardcoded
    m = 0.1 # in kg
    tau = m/alpha
    vylim = -g*m/alpha 
    
    #No analytical solution for solving Tv, so we solve numerically with fsolve
    def flytime(t):
        F = vertical_position + vylim*t + tau*(vertical_v-vylim)*(1-np.exp(-t/tau))
        return F
    
    tguess,_ = hit_conditions(vertical_position, horizontal_v, vertical_v, g) # one has to pass an initial guess to fsolve...
    Tv = fsolve(flytime,tguess)[0] #fsolve always returns a list, even with only 1 variable
    
    D = x0 + tau*horizontal_v * (1-np.exp(-Tv/tau))
    
    return Tv, D


In [8]:
def check_user_value():
    # I don't catch exceptions so that the users can see the traceback
    error_string = "YOUR FUNCTION DOES NOT SEEM RIGHT, PLEASE TRY TO FIX IT"
    ok_string = "YOUR FUNCTION SEEMS TO BE CORRECT!! CONGRATULATIONS!"
    
    test_table = []
    last_exception = None
    type_warning = False
    
    check_function_output.clear_output(wait=True)
    with check_function_output:
        user_function = code_widget.get_function_object() 

        test_values_vpos = range(1,7)
        test_values_vx = range(-2,3)
        test_values_vy = range(-2,3)
        test_values_g =  range (1,30,3)
        for test_vpos in test_values_vpos:
            for test_vx in test_values_vx:
                for test_vy in test_values_vy:
                    for test_g in test_values_g:
                        correct_value = hit_conditions(vertical_position=test_vpos, 
                            horizontal_v=test_vx,
                            vertical_v=test_vy,
                            g=test_g
                        )[1] # [1] because this gives D ([0] is instead t_hit)
                        try:
                            user_hit_position = user_function(
                                vertical_position=test_vpos, 
                                horizontal_v=test_vx,
                                vertical_v=test_vy,
                                g=test_g
                            )
                            try:
                                error = abs(user_hit_position - correct_value)
                            except Exception:
                                type_warning = True
                                error = 1. # Large value so it triggers a failed test
                        except Exception as exc:
                            last_exception = exc
                            test_table.append([test_vpos, test_vx, test_vy, test_g, correct_value, "ERROR", False])
                        else:
                            if error > 1.e-8:
                                test_table.append([test_vpos, test_vx, test_vy, test_g, str(correct_value), str(user_hit_position), False])
                            else:
                                test_table.append([test_vpos, test_vx, test_vy, test_g, str(correct_value), str(user_hit_position), True])

        num_tests = len(test_table)
        num_passed_tests = len([test for test in test_table if test[6]])
        failed_tests = [test[:-1] for test in test_table if not test[6]] # Keep only failed tests, and remove last column
        MAX_FAILED_TESTS = 5
        if num_passed_tests < num_tests:
            html_table = HTML("<style>tbody tr:nth-child(odd) { background-color: #e2f7ff; } th { background-color: #94cae0; min-width: 50px; } td { font-family: monospace; } td, th { padding-right: 10px; padding-left: 10px; }, td </style>" + 
                             tabulate.tabulate(
                                 failed_tests[:MAX_FAILED_TESTS], 
                                 tablefmt='html',
                                 headers=["hauteur initiale", "v0_x", "v0_y", "Gravité" ,"Valeur attendue", "Ta valeur"]
                             ))
                
        if num_passed_tests < num_tests:
            print("Ta fonction semble incorrecte. Seuls {}/{} tests ont pu être validés".format(num_passed_tests, num_tests))
            print("Affichage de {} des tests qui ont échoué:".format(MAX_FAILED_TESTS))
            display(html_table)
        else:
            print("Ta fonction est correcte, bravo ! Tous les {} tests ont été validés".format(num_tests))
        
        if type_warning:
            print("Attention ! Dans au moins un cas,  ta fonction n'a pas renvoyé un nombre flottant, revérifie ta syntaxe.")
    
        # Raise the last exception obtained
        if last_exception is not None:
            print("I obtained at least one exception")
            raise last_exception from None
                        
def get_user_value():
    """
    This function returns the value computed by the user's
    function for the current sliders' value, or None if there is an exception
    """
    with check_function_output:
        user_function = code_widget.get_function_object() 
        try:
            user_hit_position = user_function(
                vertical_position=vertical_position_widget.value, 
                horizontal_v=horizontal_v_widget.value,
                vertical_v=vertical_v_widget.value,
                g=g_widget.value
            )
        except Exception as exc:
            return None
    return user_hit_position        

In [9]:
with plot_box:
    the_figure = pl.figure(figsize=(6,4))
    the_plot = the_figure.add_subplot(1,1,1)

def replot(vertical_position, horizontal_v, vertical_v, g, alpha):
    global the_plot
    
    # Compute correct values
    t_hit, D = hit_conditions(vertical_position, horizontal_v, vertical_v, g)
    t_array = np.linspace(0,t_hit, 100)
    x_array, y_array = trajectory(t_array, vertical_position, horizontal_v, vertical_v, g)
    if (alpha!=0):
        t_hit_friction, D_friction = hit_conditions_friction(vertical_position, horizontal_v, vertical_v, g, alpha)
        t_array_friction = np.linspace(0,t_hit_friction, 100)
        x_array_friction, y_array_friction = trajectory_friction(t_array_friction, vertical_position, horizontal_v, vertical_v, g, alpha)

    # Clean up the graph
    the_plot.axes.clear()
    # Plot orrect curves and points
    the_plot.plot([0], [vertical_position], 'ok')
    the_plot.plot([D], [0], 'xb')    
    the_plot.plot(x_array, y_array, '-b',label='Sans frottement')
    if (alpha!=0):
        the_plot.plot([D_friction], [0], 'xk') 
        the_plot.plot(x_array_friction, y_array_friction, '--k', label='Avec frottements')

    
    ## (Try to) plot user value
    user_value = None
    try:
        user_value = get_user_value()
    except Exception:
        # Just a guard not to break the visualization, we should not end up here
        pass 
    try:
        if user_value is not None:
            the_plot.plot([user_value], [0], 'or')    
    except Exception:
        # We might end up here if the function does not return a float value
        pass 

    the_plot.axhline(0, color='gray')
    # Set zoom to fixed value
    the_plot.set_xlim([-30, 30])
    the_plot.set_ylim([-1, 16])
    # Set legends
    the_plot.set_xlabel("x [m]")
    the_plot.set_ylabel("y [m]")
    the_plot.legend()
    
    # Redraw
    the_figure.canvas.draw()
    the_figure.canvas.flush_events()

In [10]:
def recompute(e):
    global the_plot
    
    if e is not None:
        if e['type'] != 'change' or e['name'] not in ['value', 'function_body']:
            return     
    replot(
        vertical_position=vertical_position_widget.value, 
        horizontal_v=horizontal_v_widget.value,
        vertical_v=vertical_v_widget.value,
        g=g_widget.value,
        alpha = friction_widget.value
    )
    
    # Print info on the "correctness" of the user's function
    check_user_value()
    
# Bind the sliders to the event
vertical_position_widget.observe(recompute)
horizontal_v_widget.observe(recompute)
vertical_v_widget.observe(recompute)
g_widget.observe(recompute)
friction_widget.observe(recompute)

# Bind also the code widget
code_widget.observe(recompute)

In [11]:
# Perform the first recomputation (to create the plot)
_ = recompute(None)